# Searching Algorithm
- categories: [Python, SearchingAlgorithm]

### Defining a searching problem

- can either "walk" or "tram" on each state
- when choose "walk", count add 1 and cost add 1
- when choose "tram", count multiply by 2 and cost add 2

In [1]:
# define a game class to return state and all actions at that state
class CountGame:
    def __init__(self, goal):
        self.goal = goal
        
    def state(self):
        return self.goal
    
    def is_end(self, state):
        return state == self.goal
    
    def succ_and_cost(self, state):
        output = []
        if state <= self.goal // 2:
            output.append(('tram', state * 2, 2))
        if state < self.goal:
            output.append(('walk', state + 1, 1))
        return output

### recursion method - brute force

In [2]:
import math

In [3]:
def recursion(problem):
    result = {
        'cost': math.inf,    # we can also use a very large number (1,000,000 for example) instead
        'history': []
    }
    def recurse(state, history, total_cost):
        if problem.is_end(state):    # return if reach end state
            if total_cost < result['cost']:    # update best result if total cost is lower
                result['cost'] = total_cost
                result['history'] = history
            return
        for action, new_state, cost in game.succ_and_cost(state):
            next_history = history + [(action, new_state, total_cost + cost)]
            recurse(new_state, next_history, total_cost + cost)
    
    recurse(1, [], 0)
    return result

In [4]:
%%time
game = CountGame(200)
recursion(game)

CPU times: user 6.65 s, sys: 18.8 ms, total: 6.67 s
Wall time: 6.69 s


{'cost': 15,
 'history': [('walk', 2, 1),
  ('walk', 3, 2),
  ('tram', 6, 4),
  ('tram', 12, 6),
  ('tram', 24, 8),
  ('walk', 25, 9),
  ('tram', 50, 11),
  ('tram', 100, 13),
  ('tram', 200, 15)]}

It takes more than 6 seconds to search for best path towards 200...

### Dynamic programming

In [5]:
# introducing "future_cost" that depend on state only
def dynamic_programming(problem):
    cashe = {}    # store calculated items in cashe to save computation time
    result = {
        'cost': math.inf,
        'history': []
    }
    def future_cost(state):
        if cashe.get(state):
            return cashe[state]
        if problem.is_end(state):
            return 0, []
        # minor change to trace action history and store in cashe
        tot_cost = math.inf
        for action, new_state, cost in problem.succ_and_cost(state):
            f_cost, new_history = future_cost(new_state)
            if cost + f_cost < tot_cost:
                tot_cost = cost + f_cost
                next_action = [(action, new_state, tot_cost)] + new_history
        cashe[state] = tot_cost, next_action
        # original code in lecutre video, action history is not output
        # cashe[state] = min(cost + future_cost(new_state) \
        #                    for action, new_state, cost in problem.succ_and_cost(state))
        return cashe[state]
    result['cost'], result['history'] = future_cost(1)
    return result

In [6]:
%%time
game = CountGame(1000)
dynamic_programming(game)

CPU times: user 4.49 ms, sys: 1.07 ms, total: 5.56 ms
Wall time: 5.3 ms


{'cost': 22,
 'history': [('walk', 2, 22),
  ('walk', 3, 21),
  ('tram', 6, 20),
  ('walk', 7, 18),
  ('tram', 14, 17),
  ('walk', 15, 15),
  ('tram', 30, 14),
  ('walk', 31, 12),
  ('tram', 62, 11),
  ('tram', 124, 9),
  ('walk', 125, 7),
  ('tram', 250, 6),
  ('tram', 500, 4),
  ('tram', 1000, 2)]}

much faster. But will exceed the recursion limit when search up to 10,000

### Uniform search

Define data structure to store the frontier and explored nodes

In [25]:
class UniformSearchFrontier:
    def __init__(self):
        self.frontier = []
        
    def update(self, previous_state, action, new_state, cost):
        new_frontier = []
        for ps, a, ns, c in self.frontier:
            if ns != new_state:
                new_frontier.append((ps, a, ns, c))
                continue
            if c <= cost:    # if cost is larger than the cost already in frontier => do nothing
                return 

        self.frontier = new_frontier

        # else pop the state from original frontier and insert new state
        cost_list = [x[3] for x in self.frontier]
        insert_pos = self.find_insert_index(cost, cost_list)
        self.frontier = self.frontier[:insert_pos] + [(previous_state, action, new_state, cost)] + self.frontier[insert_pos:]

    def find_insert_index(self, cost, sorted_list):    # binary search pos
        if len(sorted_list) == 1:
            return 1 if sorted_list[0] > cost else 0
        if len(sorted_list) == 0:
            return 0
        index = len(sorted_list) // 2
        if sorted_list[index] > cost:
            return index + self.find_insert_index(cost, sorted_list[index:])
        else:
            return self.find_insert_index(cost, sorted_list[:index])

    def pop_min_item(self):
        if not self.frontier:
            raise ValueError('No frontier left.')
        return self.frontier.pop()

With the frontier data structure defined, the rest is quite straight forward.

In [26]:
def uniform_search(problem):
    explored = {}
    frontier = UniformSearchFrontier()
    frontier.update(0, 'start', 1, 0)
    while True:
        previous_state, action, state, tot_cost = frontier.pop_min_item()
        explored[state] = (previous_state, tot_cost, action)
        if problem.is_end(state):
            break
        for action, new_state, cost in problem.succ_and_cost(state):
            if new_state in explored:
                continue
            frontier.update(state, action, new_state, tot_cost + cost)
        
    result = []
    s = problem.goal
    while explored.get(s):
        result.append((explored[s][2], s, explored[s][1]))
        s = explored[s][0]
    return result

In [27]:
%%time
game = CountGame(1000)
uniform_search(game)

CPU times: user 37.4 ms, sys: 1.72 ms, total: 39.1 ms
Wall time: 37.9 ms


[('tram', 1000, 22),
 ('tram', 500, 20),
 ('tram', 250, 18),
 ('walk', 125, 16),
 ('tram', 124, 15),
 ('tram', 62, 13),
 ('walk', 31, 11),
 ('tram', 30, 10),
 ('walk', 15, 8),
 ('tram', 14, 7),
 ('walk', 7, 5),
 ('tram', 6, 4),
 ('walk', 3, 2),
 ('walk', 2, 1),
 ('start', 1, 0)]

In this case uniform search is not restricted by recursion limit.

In [24]:
%%time
game = CountGame(100000)
uniform_search(game)

CPU times: user 2min 51s, sys: 1.35 s, total: 2min 52s
Wall time: 2min 53s


[('tram', 100000, 36),
 ('tram', 50000, 34),
 ('tram', 25000, 32),
 ('tram', 12500, 30),
 ('tram', 6250, 28),
 ('walk', 3125, 26),
 ('tram', 3124, 25),
 ('tram', 1562, 23),
 ('walk', 781, 21),
 ('tram', 780, 20),
 ('tram', 390, 18),
 ('walk', 195, 16),
 ('tram', 194, 15),
 ('walk', 97, 13),
 ('tram', 96, 12),
 ('tram', 48, 10),
 ('tram', 24, 8),
 ('tram', 12, 6),
 ('tram', 6, 4),
 ('walk', 3, 2),
 ('walk', 2, 1),
 ('start', 1, 0)]

reference: 
- <a href="https://www.youtube.com/watch?v=aIsgJJYrlXk&ab_channel=stanfordonline">Stanford CS221: AI (Autumn 2019)</a>